# ATR(Average True Range)기반 변동성 돌파 매매 전략

In [31]:
!pip install backtrader
!pip install yfinance

In [ ]:
import backtrader as bt
import math
import yfinance as yf

In [37]:
import backtrader as bt
import datetime

import backtrader as bt

# 변동성 돌파 전략 (ATR 기반)
class VolatilityBreakout(bt.Strategy):
    params = dict(
        atr_period=14,       # ATR 기간
        risk_percent=0.2,    # 손절 비율
        profit_percent=0.2,  # 익절 비율
    )

    def __init__(self):
        # ATR 지표 계산
        self.atr = bt.indicators.ATR(self.data, period=self.p.atr_period)

        # 주문 상태 및 매수/손절 관련 변수 초기화
        self.order = None
        self.buy_price = None
        self.last_date = None
        self.entered_today = False

    def next(self):
        # 현재 날짜 확인
        current_date = self.data.datetime.date(0)

        # 날짜가 바뀌면 하루 목표 가격과 진입 상태를 초기화
        if self.last_date != current_date:
            self.target_price = self.data.open[0] + self.atr[0]
            self.entered_today = False
            self.last_date = current_date

        # 주문 처리 중이면 대기
        if self.order:
            return

        # 포지션이 없고 오늘 미진입 상태일 때
        if not self.position and not self.entered_today:
            # 목표가 돌파 시 매수
            if self.data.high[0] >= self.target_price:
                self.buy_price = self.target_price
                self.order = self.buy()    # 전액 시장가 매수
                self.entered_today = True

        # 포지션 보유 중일 때 손절/익절 체크
        elif self.position:
            current_price = self.data.close[0]

            # 익절 조건
            if current_price >= self.buy_price * (1 + self.p.profit_percent):
                self.order = self.sell()  # 전량 시장가 매도

            # 손절 조건
            elif current_price <= self.buy_price * (1 - self.p.risk_percent):
                self.order = self.sell()  # 전량 시장가 매도

    def log(self, message):
        print(message)

    def notify_order(self, order):
        # 주문 제출/수락 상태 처리 없음
        if order.status in [order.Submitted, order.Accepted]:
            return

        cur_date = self.data.datetime.date(0)
        # 주문 체결 확인
        if order.status == order.Completed:
            if order.isbuy():
                # 매수 체결 로그 및 매수 가격 재설정
                self.log(f'{cur_date} [매수 주문 실행] 종목: {order.data._name}  \t 수량: {order.size}  \t 가격: {order.executed.price:.2f}')
                self.buy_price = order.executed.price

            elif order.issell():
                # 매도 체결 로그 및 매수 가격 초기화
                self.log(f'{cur_date} [매도 주문 실행] 종목: {order.data._name}  \t 수량: {order.size}  \t 가격: {order.executed.price:.2f}')
                self.buy_price = None

        # 주문 거부/취소/마진 부족 처리
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name}  \t 수량: {order.size}')

        # 모든 경우에 주문 플래그 초기화
        self.order = None



In [38]:
if __name__ == '__main__':
    import yfinance as yf

    cerebro = bt.Cerebro()
    cerebro.addstrategy(VolatilityBreakout)

    # 초기 자본 및 수수료 세팅
    cerebro.broker.setcash(10_000_000)
    cerebro.broker.setcommission(commission=0.003)

    # 데이터 로드
    data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)

    print('Starting Portfolio Value:', cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


[*********************100%***********************]  1 of 1 completed


Starting Portfolio Value: 10000000
2020-01-28 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 8912.52
2020-03-13 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 5017.83
2020-03-14 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 5573.08
2020-03-25 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 6738.72
2020-03-31 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 6430.61
2020-04-28 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 7796.97
2020-04-30 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 8797.67
2020-07-28 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 11017.46
2020-07-30 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 11099.83
2020-10-28 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 13654.21
2020-11-01 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 13781.00
2020-11-17 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 16685.69
2020-11-18 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 17645.19
2020-12-17 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 21308.35
2020-12-18 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 22806.80
2020-12-31 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 28841.57
2021-01-03 [매수 주문 실행] 종목:   	 수량: 1  	 가격: 32129.41
2021-01-08 [매도 주문 실행] 종목:   	 수량: -1  	 가격: 39381.77
2021-01-09 [매수 주문 실행] 종목:  

In [39]:
import backtrader as bt
import yfinance as yf
import datetime

# 리스크 고정 투자법 전략
class VolatilityBreakoutFixedRiskStrategy(bt.Strategy):
    params = (
        ('risk_percent', 0.02),  # 리스크 비율 (2%)
        ('atr_period', 14),  # ATR 계산 기간
        ('atr_multiplier', 2),  # ATR 곱하여 손절 가격 설정
    )

    def __init__(self):
        # ATR 계산 (변동성을 기준으로 손절 가격 설정)
        self.atr = bt.indicators.ATR(self.data, period=self.p.atr_period)

        # 주문 상태 및 매수/손절 관련 변수 초기화
        self.order = None  # 주문 상태 초기화
        self.buy_price = None  # 초기 매수 가격 (초기화 추가)

    def calculate_position_size(self, entry_price, stop_loss_price):
        """포지션 크기 계산: 리스크 고정 투자법 적용"""
        # 계좌에서 허용되는 리스크 금액
        risk_amount = self.broker.getvalue() * self.p.risk_percent
        # 한 주당 손실
        per_unit_risk = abs(entry_price - stop_loss_price)
        # 포지션 크기 계산
        if per_unit_risk > 0:
            position_size = risk_amount / per_unit_risk
            return int(position_size)  # 소수점 제거
        return 0

    def next(self):
        if self.order:  # 주문이 처리 중이면 대기
            return

        if not self.position:  # 포지션이 없는 경우
            entry_price = self.data.close[0]
            stop_loss_price = entry_price - (self.atr[0] * self.p.atr_multiplier)

            # 목표 가격 돌파 시 매수
            if self.data.close[0] > self.data.open[0]:  # 간단한 상승 신호
                position_size = self.calculate_position_size(entry_price, stop_loss_price)
                if position_size > 0:
                    self.buy_price = entry_price  # 매수 가격 기록
                    self.order = self.buy(size=position_size)
        else:  # 포지션이 있는 경우
            current_price = self.data.close[0]

            # 손절 조건: 현재 가격이 손절 가격보다 낮아지면 매도
            if current_price < self.buy_price - (self.atr[0] * self.p.atr_multiplier):
                self.order = self.sell(size=self.position.size)

    def notify_order(self, order):
        if order.status in [order.Completed, order.Canceled, order.Rejected]:
            self.order = None  # 주문 완료 후 초기화


In [40]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(VolatilityBreakoutFixedRiskStrategy)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


Final Portfolio Value: 23244720.01
수익률: 232.45%
